# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Task 2c.2 - Implementation of a Lagrangian Neural Network (LNN) (9p)

**Author:** Maximilian Stölzle (M.W.Stolzle@tudelft.nl)

This notebook will guide you through the implementation of a Lagrangian neural network.

In [ ]:
# Reloads the python files outside of this notebook automatically
%load_ext autoreload
%autoreload 2

# import all Python modules
from distutils.util import strtobool
from functools import partial
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
import jax
from jax import random
from jax import numpy as jnp
import os
from pathlib import Path

from jax_double_pendulum.integrators import rk4_step
from jax_double_pendulum.utils import normalize_link_angles

# define boolean to check if the notebook is run for the purposes of autograding
AUTOGRADING = strtobool(os.environ.get("AUTOGRADING", "false"))

## Neural networks (2p)

First, please finish the implementation of the neural networks `MassMatrixNN` and `PotentialEnergyNN`. Both neural networks should consist of four layers and have the following structure:

1. A linear layer mapping from the input dimension to the hidden state dimension $n_\mathrm{hidden}$.

2. A softplus activation function.

3. A linear layer with $n_\mathrm{hidden}$ as both the input and output dimension.

4. A softplus activation function.

5. A linear layer with $n_\mathrm{hidden}$ as both the input and output dimension.

6. A softplus activation function.

7. A linear layer with $n_\mathrm{hidden}$ as both the input and output dimension.

8. A softplus activation function.

9. A linear layer mapping from $n_\mathrm{hidden}$ to the output dimension.


The hidden state dimension is set by the `num_hidden` attribute of the class to $32$. The softplus activation function is.a smooth approximation of the Rectified Linear Unit (ReLU) function.

The mass matrix $M$ (sometimes also denoted as $B$) of mechanical systems has two important characteristics, which we want to enforce for our learned mass matrix as well:

- The diagonal values are always positive
- The matrix is symmetric

The symmetry is given automatically by the fact that we only learn a triangular matrix and then derive the complete mass matrix from that triangular matrix. We can induce the first characteristic by applying a softplus activation function in conjunction with a shifting of the diagonals.

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# import MassMatrixNN and PotentialEnergyNN from lnn.ipynb
from ipynb.fs.full.lnn import MassMatrixNN, PotentialEnergyNN

# initialize the PRNG key at seed 0
_rng = random.PRNGKey(seed=0)

# initialize parameters of MassMatrixNN by passing a dummy input of ones through the network
_mass_matrix_nn_params = MassMatrixNN().init(_rng, jnp.ones((2,)))["params"]

# initialize parameters of PotentialEnergyNN by passing a dummy input of ones through the network
_potential_energy_nn_params = PotentialEnergyNN().init(_rng, jnp.ones((2,)))["params"]

# some dummy system state
_th = 0 * jnp.ones((2,))
_th_d = jnp.pi * jnp.ones((2,))

_M_nn = MassMatrixNN().apply({"params": _mass_matrix_nn_params}, _th)
_U = PotentialEnergyNN().apply({"params": _potential_energy_nn_params}, _th)

print("Computed mass matrix (output from neural network):\n", _M_nn)
print("Computed potential energy:\n", _U)


## Lagrangian (1p)

First, please implement the two functions `kinetic_energy_fn` and `potential_energy_fn` which compute the (estimated) kinetic energy and potential energy of the system respectively:

\begin{equation}
    \mathcal{T} = \frac{1}{2} \: \dot{\theta}^\mathrm{T} \: f_\mathrm{M}(\theta, \pi_\mathrm{M}) \: \dot{\theta}, 
    \qquad 
    \mathcal{U} = f_\mathcal{U}(\theta, \pi_\mathcal{U}),
\end{equation}

where $f_\mathrm{M}(\theta, \pi_\mathrm{M})$ and $f_\mathcal{U}(\theta, \pi_\mathcal{U})$ are the two neural network forward functions respectively.

The Lagrangian $\mathcal{L}$ is then defined as

\begin{equation}
    \mathcal{L} = \mathcal{T} - \mathcal{U},
\end{equation}

and needs to be implemented into `lagrangian_fn`.

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# import lagrangian_fn from lnn.ipynb
from ipynb.fs.full.lnn import lagrangian_fn

_L = lagrangian_fn(
    _mass_matrix_nn_params,
    _potential_energy_nn_params,
    _th,
    _th_d,
)
print("Computed Lagrangian:", _L)


## Continuous forward dynamics (4p)

Next, please take the various partial derivatives of the Lagrangian to compute the dynamical matrices (i.e. the components of the equations of motion) in the function `dynamical_matrices`

\begin{equation}
    M = \frac{\partial^2 \mathcal{L}}{\partial \dot{\theta}^2},
    \qquad
    C = \frac{\partial^2 \mathcal{L}}{\partial \theta \partial \dot{\theta}},
    \qquad
    G = - \frac{\partial \mathcal{L}}{\partial \theta}.
\end{equation}

Make sure that before you evaluate the Lagrangian, you normalize the link angles $\theta$ to the interval $[-\pi, \pi]$. If we were not to implement this normalization, we would (likely) get a different behaviour for $\theta = a$ or $\theta = a + 2 \: \pi$, which is not desired. This would also drastically increase the amount of training data we would require.

Then, please compute the continuous forward dynamics (i.e. determining $\ddot{\theta}$ for a given state $(\theta, \dot{\theta})$ and input $\tau$) in the function `continuous_forward_dynamics`. For this implementation, you can take inspiration from the analytical solution in `jax_double_pendulum.dynamics.continuous_forward_dynamics`.

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# import dynamical_matrices from lnn.ipynb
from ipynb.fs.full.lnn import dynamical_matrices, continuous_forward_dynamics

_M, _C, _G = dynamical_matrices(
    _mass_matrix_nn_params,
    _potential_energy_nn_params,
    _th,
    _th_d,
)

print("Computed M:\n", _M)
print("Computed C:\n", _C)
print("Computed G:\n", _G)

# zero applied torque
_tau = jnp.ones((2,))

_th_dd = continuous_forward_dynamics(
    _mass_matrix_nn_params,
    _potential_energy_nn_params,
    _th,
    _th_d,
    _tau,
)

print("Computed th_dd:\n", _th_dd)

# the mass matrix estimated by the neural network should be the same as d2L_d2thd
assert jnp.allclose(_M_nn, _M)


## State-space dynamics (1p)

Please implement dynamics in a nonlinear state-space representation into the function `continuous_state_space_dynamics`. The state-space representation needs to have the following structure

\begin{equation}
    \frac{\mathrm{d}x}{\mathrm{d}t} = f(x, \tau),\\
    y = g(x, \tau),
\end{equation}

where $x \in \mathbb{R}^4$ is the state of the system. For implementing this function, you can take inspiration from `jax_double_pendulum.dynamics.continuous_state_space_dynamics`.

Having the EoM in this structure will allow us in the next step to use standard methods such as [RK4](https://en.wikipedia.org/wiki/Runge–Kutta_methods) to integrate the ODE in time.

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# import continuous_state_space_dynamics from lnn.ipynb
from ipynb.fs.full.lnn import continuous_state_space_dynamics

# define a dummy state
_x = jnp.concatenate([_th, _th_d])

_dx_dt, _y = continuous_state_space_dynamics(
    _mass_matrix_nn_params,
    _potential_energy_nn_params,
    _x,
    _tau,
)

print("Time derivative of the state\n", _dx_dt)
print("System output", _y)


## Discrete-time forward dynamics (1p)

Now, we move on to implementing the discrete-time forward dynamics into `discrete_forward_dynamics`.
Please use the `jax_double_pendulum.integrators.rk4_step` function to integrate the continuous state-space dynamics for one time-step of duration `dt`. The `rk4_step` expects as its first argument an `ode_fun`, which needs to conform to the following syntax: `rk4_step(x) -> dx_dt` (i.e. a mapping from the state $x$ to its time-derivative $\dot{x}$). For implementing this function, you can take inspiration from `jax_double_pendulum.dynamics.discrete_forward_dynamics`.

In [ ]:
# DO NOT REMOVE OR MODIFY THIS CELL

# import discrete_forward_dynamics from lnn.ipynb
from ipynb.fs.full.lnn import discrete_forward_dynamics

_dt = 1e-2  # time-step [s]
_th_next, _th_d_next, _th_dd = discrete_forward_dynamics(
    _mass_matrix_nn_params,
    _potential_energy_nn_params,
    _dt,
    _th,
    _th_d,
    _tau,
)

print("Computed next link angles:", _th_next)
print("Computed next link angular velocities:", _th_d_next)
print("Computed angular link accelerations:", _th_dd)

